In [1]:
import wrangle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df = wrangle.get_zillow()
df = wrangle.prep_zillow(df)
df = wrangle.remove_outliers(df)

In [3]:
df.

AttributeError: 'DataFrame' object has no attribute 'doll'